<a href="https://colab.research.google.com/github/richardkilea/Neo4J/blob/main/Knowledge_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook shows how you can query a database

In [ ]:
from IPython.display import clear_output

!pip install python-docx
!pip install python-dotenv
!pip install langchain
!pip install langchainhub
!pip install openai langchain-openai
!pip install -U langchain-openai
!pip install langchain_community
!pip install neo4j

In [6]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [9]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('URI')
NEO4J_USERNAME = os.getenv('neo4j')
NEO4J_PASSWORD = os.getenv('PASSWORD')
graph = Neo4jGraph()

Initialize a knowledge graph instance using LangChain's Neo4j integration

In [10]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

#Querying the movie knowledge graph
Match all nodes in the graph

In [11]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """

In [12]:
result = kg.query(cypher)
result

[{'count(n)': 171}]

In [13]:
cypher = """
  MATCH (n)
  RETURN count(n) AS numberOfNodes
  """

In [14]:
result = kg.query(cypher)
result

[{'numberOfNodes': 171}]

In [15]:
print(f"There are {result[0]['numberOfNodes']} nodes in this graph.")

There are 171 nodes in this graph.


Match only the Movie nodes by specifying the node label

In [16]:
cypher = """
  MATCH (n:Movie)
  RETURN count(n) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 38}]

- Change the variable name in the node pattern match for improved readability

In [17]:
cypher = """
  MATCH (m:Movie)
  RETURN count(m) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 38}]

Match only the Person nodes

In [18]:
cypher = """
  MATCH (people:Person)
  RETURN count(people) AS numberOfPeople
  """
kg.query(cypher)

[{'numberOfPeople': 133}]

Match a single person by specifying the value of the name property on the Person node

In [19]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})
  RETURN tom
  """
kg.query(cypher)

[{'tom': {'born': 1956, 'name': 'Tom Hanks'}}]

Match a single Movie by specifying the value of the title property

In [20]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"})
  RETURN cloudAtlas
  """
kg.query(cypher)

[{'cloudAtlas': {'tagline': 'Everything is connected',
   'title': 'Cloud Atlas',
   'released': 2012}}]

Return only the released property of the matched Movie node

In [21]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"})
  RETURN cloudAtlas.released
  """
kg.query(cypher)

[{'cloudAtlas.released': 2012}]

Return two properties

In [22]:
cypher = """
  MATCH (cloudAtlas:Movie {title:"Cloud Atlas"})
  RETURN cloudAtlas.released, cloudAtlas.tagline
  """
kg.query(cypher)

[{'cloudAtlas.released': 2012,
  'cloudAtlas.tagline': 'Everything is connected'}]

#Cypher patterns with conditional matching

In [23]:
cypher = """
  MATCH (nineties:Movie)
  WHERE nineties.released >= 1990
    AND nineties.released < 2000
  RETURN nineties.title
  """


In [24]:
kg.query(cypher)

[{'nineties.title': 'Joe Versus the Volcano'},
 {'nineties.title': 'A Few Good Men'},
 {'nineties.title': 'Unforgiven'},
 {'nineties.title': 'Hoffa'},
 {'nineties.title': 'A League of Their Own'},
 {'nineties.title': 'Sleepless in Seattle'},
 {'nineties.title': 'Johnny Mnemonic'},
 {'nineties.title': 'Apollo 13'},
 {'nineties.title': 'That Thing You Do'},
 {'nineties.title': 'The Birdcage'},
 {'nineties.title': 'Twister'},
 {'nineties.title': "The Devil's Advocate"},
 {'nineties.title': 'As Good as It Gets'},
 {'nineties.title': 'What Dreams May Come'},
 {'nineties.title': "You've Got Mail"},
 {'nineties.title': 'When Harry Met Sally'},
 {'nineties.title': 'The Matrix'},
 {'nineties.title': 'Snow Falling on Cedars'},
 {'nineties.title': 'The Green Mile'},
 {'nineties.title': 'Bicentennial Man'}]

#Pattern matching with multiple nodes

In [25]:
cypher = """
  MATCH (actor:Person)-[:ACTED_IN]->(movie:Movie)
  RETURN actor.name, movie.title LIMIT 10
  """
kg.query(cypher)

[{'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix'},
 {'actor.name': 'Emil Eifrem', 'movie.title': 'The Matrix'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Carrie-Anne Moss', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Laurence Fishburne', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Hugo Weaving', 'movie.title': 'The Matrix Reloaded'},
 {'actor.name': 'Keanu Reeves', 'movie.title': 'The Matrix Revolutions'}]

In [26]:
cypher = """
  MATCH (tom:Person {name: "Tom Hanks"})-[:ACTED_IN]->(tomHanksMovies:Movie)
  RETURN tom.name,tomHanksMovies.title
  """
kg.query(cypher)

[{'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "You've Got Mail"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Sleepless in Seattle'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Joe Versus the Volcano'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'That Thing You Do'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cloud Atlas'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Da Vinci Code'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Green Mile'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Apollo 13'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'Cast Away'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': "Charlie Wilson's War"},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'The Polar Express'},
 {'tom.name': 'Tom Hanks', 'tomHanksMovies.title': 'A League of Their Own'}]

In [27]:
cypher = """
  MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors)
  RETURN coActors.name, m.title
  """
kg.query(cypher)

[{'coActors.name': 'Meg Ryan', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Greg Kinnear', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Parker Posey', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Dave Chappelle', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Steve Zahn', 'm.title': "You've Got Mail"},
 {'coActors.name': 'Meg Ryan', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Rita Wilson', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Bill Pullman', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Victor Garber', 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': "Rosie O'Donnell", 'm.title': 'Sleepless in Seattle'},
 {'coActors.name': 'Meg Ryan', 'm.title': 'Joe Versus the Volcano'},
 {'coActors.name': 'Nathan Lane', 'm.title': 'Joe Versus the Volcano'},
 {'coActors.name': 'Charlize Theron', 'm.title': 'That Thing You Do'},
 {'coActors.name': 'Liv Tyler', 'm.title': 'That Thing You Do'},
 {'coActors.name': 'Hugo Weaving', 'm.title

#Delete data from the graph

In [28]:
cypher = """
MATCH (emil:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
RETURN emil.name, movie.title
"""
kg.query(cypher)

[{'emil.name': 'Emil Eifrem', 'movie.title': 'The Matrix'}]

In [29]:
cypher = """
MATCH (emil:Person {name:"Emil Eifrem"})-[actedIn:ACTED_IN]->(movie:Movie)
DELETE actedIn
"""
kg.query(cypher)

[]

#Adding data to the graph

In [30]:
cypher = """
CREATE (andreas:Person {name:"Andreas"})
RETURN andreas
"""

kg.query(cypher)

[{'andreas': {'name': 'Andreas'}}]

In [31]:
cypher = """
MATCH (andreas:Person {name:"Andreas"}), (emil:Person {name:"Emil Eifrem"})
MERGE (andreas)-[hasRelationship:WORKS_WITH]->(emil)
RETURN andreas, hasRelationship, emil
"""
kg.query(cypher)

[{'andreas': {'name': 'Andreas'},
  'hasRelationship': ({'name': 'Andreas'},
   'WORKS_WITH',
   {'born': 1978, 'name': 'Emil Eifrem'}),
  'emil': {'born': 1978, 'name': 'Emil Eifrem'}}]